Run the following sequence for all answers

In [1]:
import numpy as np
import math

The following class is for answer 1

In [5]:
class Answer1:

    # p is the end effector position
    # d1 = link 1 length
    # a2 = link 2 length
    def __init__(self) -> None:
        self.p = []
        self.d1 = 0
        self.a2 = 0
        self.theta_1 = 0
        self.theta_2 = 0
        self.d3 = 0
        pass

    def takeInputs(self):
        p = [eval(j) for j in input("Enter the end effector position in x y z format" + " "*6).split()]
        self.p = p
        d1 = float(input("Length of link 1: "))
        self.d1 = d1
        a2 = float(input("Length of link 2: "))
        self.a2 = a2
        self.inverseKinematics()
        pass

    def inverseKinematics(self):
        px = self.p[0]
        py = self.p[1]
        pz = self.p[2]
        theta_1 = math.atan((px/py))
        r = math.sqrt((px**2) + (py**2))
        s = pz - self.d1
        theta_2 = math.atan((r/s))
        d3 = math.sqrt((r**2)+(s**2))-self.a2

        self.theta_1 = theta_1
        self.theta_2 = theta_2
        self.d3 = d3
        pass

    def callback(self):
        self.takeInputs()
        return self.theta_1, self.theta_2, self.d3

a1 = Answer1()
theta1, theta2, d3 = a1.callback()
print(theta1, theta2, d3)

0.4636476090008061 1.1502619915109316 -0.5505102572168217


The following class is for answer 2

In [ ]:
class Answer2:

    # R and d are given
    # a1 = link 1 length
    # a2 = link 2 length
    # d4 = distance between o3 and o4
    def __init__(self) -> None:
        self.d = 0
        self.R = []
        self.a1 = 0
        self.a1 = 0
        self.d4 = 0
        self.theta_1 = 0
        self.theta_2 = 0
        self.theta_4 = 0
        self.d3 = 0
        pass

    def takeInputs(self):
        d = [eval(j) for j in input("Value of d as dx dy dz: ").split()]
        self.d = d
        R = np.zeros((3,3))
        for i in range(3):
            R[i,:] = [eval(j) for j in input("row " + str(i+1) + " as c1 c2 c3: ").split()]
        self.R = R
        a1 = float(input("Link 1 length: "))
        a2 = float(input("Link 2 length"))
        d4 = float(input("Distance between o3 and o4"))
        self.a1 = a1
        self.a2 = a2
        self.d4 = d4
        self.inverseKinematics()
        pass

    def inverseKinematics(self):
        dx = self.d[0]
        dy = self.d[1]
        dz = self.d[2]
        r = math.sqrt((dx**2 + dy**2 - self.a1**2 - self.a2**2)/(2*self.a1*self.a2))
        theta_2 = math.atan(math.sqrt(1 - r**2)/r)
        # other value would be atan(-sqrt(1-r^2)/r)
        theta_1 = math.atan((dx/dy)) - math.atan((self.a1 + self.a2*math.cos(theta_2))/(self.a2*math.sin(theta_2)))
        alpha = math.atan((self.R[1][2]/self.R[1][1]))
        theta_4 = theta_1+theta_2-alpha
        d3 = dz + self.d4
        self.theta_1 = theta_1
        self.theta_2 = theta_2
        self.theta_4 = theta_4
        self.d3 = d3
        pass

    def callback(self):
        self.takeInputs()
        return self.theta_1, self.theta_2, self.theta_4, self.d3

a2 = Answer2()
theta1, theta2, theta4, d_3 = a2.callback()
print(theta1, theta2, theta4, d_3)

Note: The class for calculating Jacobian matrix is based on the code written by Harsh Mandalia for the previous assignment.

In [ ]:
class Jacobian:

    def __init__(self) -> None:
        self.n = 0
        self.dh_params = []
        self.A = []
        self.o = []
        self.Z = []
        self.J = []
        self.R_or_P = []
        self.EF_pos = []
        pass

    def takeInputs(self):
        n = int(input("Number of links:"))
        dh_params = [0]*n
        R_or_P = [0]*n
        print("Link |   theta_i |   d_i |   a_i |   alpha_")
        for i in range(n):
            dh_params[i] = [eval(j) for j in input(str(i+1) + " "*6).split()]
        
        lst=[int(i) for i in input("Type the joint numbers of prismatic joints (leave empty if none): ").split()]

        for i in lst:
            R_or_P[i-1]=1
        self.n = n
        self.dh_params = dh_params
        self.A_matrix()
        self.R_or_P = R_or_P
        pass

    def A_matrix(self):
        A = [0]*self.n
        for i in range(self.n):
            theta,d,a,alpha = self.dh_params[i]
            A[i] = np.array([[np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
                             [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
                             [0, np.sin(alpha), np.cos(alpha), d],
                             [0, 0, 0, 1]])
        self.A = A
        self.o_and_z_matrix()
        pass
    
    def o_and_z_matrix(self):
        EF_pos = np.identity(4)
        Z = [0]*(self.n +1)
        Z[0] = np.array([[0],[0],[1]])
        o = [0]*(self.n+1)
        o[0] = np.array([0,0,0])

        for i in range(self.n):
            EF_pos = EF_pos@self.A[i]
            Z[i+1]=EF_pos[0:3, 0:3]@Z[0]
            o[i+1]=EF_pos[0:3,3].T
        
        EF_pos = EF_pos[0:3,3]
        self.EF_pos = EF_pos
        self.o = o
        self.Z = Z
        self.calculateJ()
        pass

    def calculateJ(self):
        J = [0]*self.n
        for i in range(self.n):
            if self.R_or_P[i] == 1:
                J[i] = np.concatenate((self.Z[i], np.array([[0,0,0]]).T), axis=0)
            else:
                cross=np.cross(self.Z[i].T,(self.o[self.n]-self.o[i]).T)
        J[i]=np.concatenate((cross.T,self.Z[i]),axis=0)

        final_J = J[0]
        for i in range(1, self.n):
            final_J = np.concatenate((final_J, J[i]), axis=1)

        self.J = final_J
        pass

    def callbackJ(self):

        self.takeInputs()
        return self.J

The following two classes are for answer 3

In [ ]:
class Answer3(Jacobian):

    def __init__(self) -> None:
        super().__init__()
        self.Xdot = []
        self.qdot = []
        pass

    def inputs(self):
        x_dots = [eval(j) for j in input("End effector cartesian velocities as x_dot y_fot z_dot").split()]
        self.Xdot = x_dots
        self.calculate_qdot()
        pass

    def calculate_qdot(self):
        J = self.callbackJ()
        J_inv = np.linalg.pinv(J)
        q_dots = J_inv@self.Xdot
        self.qdot = q_dots
        pass

    def callback_qdot(self):
        self.inputs()
        return self.qdot

a3 = Answer3()
qdot = a3.callback_qdot()
print(qdot)

The following class is for answer 6

In [ ]:
class Answer6:

    def __init__(self) -> None:
        self.U = []
        self.theta = 0
        self.phi = 0
        self.psi = 0
        pass

    def takeInputs(self):
        U = np.zeros((3,3))
        for i in range(3):
            U[i,:] = [eval(j) for j in input("Enter row" + str(i+1) + " of U as c1 c2 c3: ").split()]
        self.U = U
        self.inverseKinematics()
        pass

    def inverseKinematics(self):
        theta = 0
        phi = 0
        psi = 0

        if self.U[0][2]!=0 or self.U[1][2]!=0 :
            m = int(input("Enter 1 for sin(theta) > 0 or 0 for sin(theta) <0 :"))
            if m == 1:
                theta = math.atan((math.sqrt(1 - self.U[2][2]*self.U[2][2]))/(self.U[2][2]))
                phi = math.atan((self.U[1][2])/(self.U[0][2]))
                psi = math.atan((self.U[2][1])/(-self.U[2][0]))
            elif m == 0:
                theta = math.atan((-math.sqrt(1 - self.U[2][2]*self.U[2][2]))/(self.U[2][2]))
                phi = math.atan((-self.U[1][2])/(-self.U[0][2]))
                psi = math.atan((-self.U[2][1])/(self.U[2][0]))
            else:
                print("Enter only 0s and 1s for the value of sin(theta)")

        else:
            if self.U[2][2] == 1:
                theta = 0
                temp = math.atan((self.U[1][0])/(self.U[0][0]))
                # By convention the value of phi is given as below
                phi = 0
                psi = temp - phi
            elif self.U[2][2] == -1:
                theta = math.pi
                temp = math.atan((-self.U[0][1])/(-self.U[0][0]))
                # By convention the value of phi is given as below
                phi = 0
                psi = phi - temp
            else:
                print("Enter valid input matrix")
    
        self.theta = theta*180/math.pi
        self.phi = phi*180/math.pi
        self.psi = psi*180/math.pi

        self.callback()
        pass

    def callback(self):
        self.takeInputs()
        return [self.theta, self.phi, self.psi]

a6 = Answer6()
[theta, phi, psi] = a6.callback()
eulerAngles = [theta, phi, psi]
print(eulerAngles)

The following class is for carrying out answer 7

In [ ]:
class Answer7(Jacobian):

    def __init__(self) -> None:
        super().__init__()
        self.qdot = []
        pass

    def inputs(self):
        q_dots = [eval(j) for j in input("Enter the value for q_dot as d1_dot d2_dot d3_dot: ").split()]
        self.qdot = q_dots
        pass

    def callback(self):
        J = self.callbackJ()
        EF_pos = self.EF_pos
        self.inputs()
        q_dot = self.qdot
        EF_vel = J@q_dot
        return EF_pos, EF_vel

a7 = Answer7()
ef_pos, ef_vel = a7.callback()
print(ef_pos, ef_vel)